# CIGRE MV Powerflow Solver Test

## Run simulation

In [6]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

CIGRE_MV_PowerFlowTest ${TOP}/Examples/CIM/CIGRE_MV_NoTap/*.xml

CIMContentHandler: Note: 0 out of 778 tasks remain unresolved!


In [7]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import villas.dataprocessing.validationtools as validationtools
import os
import urllib.request

## Read DPsim results

In [8]:
path = 'logs/'
logName = 'CIGRE-MV-NoTap'
dpsim_result_file = path + logName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

# Fix for dpsim naming - TODO: unify dpsim notation in log file and update villas-dataprocessing accordingly
for ts,values in ts_dpsim.items():
    values.name = values.name[:-2]

column number: 15
results length: 120
real column names: []
complex column names: ['N0.V', 'N1.V', 'N10.V', 'N11.V', 'N12.V', 'N13.V', 'N14.V', 'N2.V', 'N3.V', 'N4.V', 'N5.V', 'N6.V', 'N7.V', 'N8.V', 'N9.V']


## Read NEPLAN results as reference

In [9]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/Neplan/ReferenceGrids/CIGRE_MV.rlf'
neplan_result_file = 'reference-results/CIGRE_MV.rlf'
urllib.request.urlretrieve(url, neplan_result_file) 
print(neplan_result_file)

ts_NEPLAN = read_timeseries_NEPLAN_loadflow(neplan_result_file)
#print([ts_NEPLAN[i].name for i in range(len(ts_NEPLAN))])

reference-results/CIGRE_MV.rlf


## Comparison and assertion of results

In [10]:
net_name='CIGRE_MV_NoTap'
threshold=0.5

ts_NEPLAN_standardized=validationtools.convert_neplan_to_standard_timeseries(ts_NEPLAN)
#print([ts_NEPLAN_standardized[i].name for i in range(len(ts_NEPLAN_standardized))])
ts_dpsim_standardized=validationtools.convert_dpsim_to_standard_timeseries(ts_dpsim)
#print([ts_dpsim_standardized[i].name for i in range(len(ts_dpsim_standardized))])
res_err=validationtools.compare_timeseries(ts_NEPLAN_standardized,ts_dpsim_standardized)
validationtools.assert_modelica_results(net_name,res_err,threshold)

N0.V
0.0
N0.VANGLE
0.0
N1.V
0.06300359551948702
N1.VANGLE
-0.0012694961243216723
N10.V
0.07428412292891705
N10.VANGLE
-0.04194141143223348
N11.V
0.07430912647323866
N11.VANGLE
-0.04302873570896224
N12.V
0.03123342083313643
N12.VANGLE
-0.006754175950550075
N13.V
0.031595742878392605
N13.VANGLE
-0.0011994603355235036
N14.V
0.03176328011913161
N14.VANGLE
-0.004045279098843493
N2.V
0.06678100763349827
N2.VANGLE
-0.012985889677551989
N3.V
0.07294404258661458
N3.VANGLE
-0.03547533321803249
N4.V
0.07326300316120848
N4.VANGLE
-0.03625517542820986
N5.V
0.07352009697457926
N5.VANGLE
-0.04357076634514172
N6.V
0.07373436305224995
N6.VANGLE
-0.042324486702345135
N7.V
0.07395537135837776
N7.VANGLE
-0.042092250576904654
N8.V
0.07393415040058174
N8.VANGLE
-0.041034305497914855
N9.V
0.07410648062830545
N9.VANGLE
-0.03531822973634549
Test on N0.V Passed
Test on N0.VANGLE Passed
Test on N1.V Passed
Test on N1.VANGLE Passed
Test on N10.V Passed
Test on N10.VANGLE Passed
Test on N11.V Passed
Test on N11.VA